# Notebook 4: Scientific Machine Learning (SciML) - Physics + AI

**Topic:** PhD Thesis - Scientific Computing & Geophysical Inversion  
**Description:** This notebook demonstrates the hybrid "Scientific Machine Learning" approach. We replace the traditional smoothing regularization with a **Restricted Boltzmann Machine (RBM)** trained on geological patterns. This allows the P-bit optimizer to recover sharp, blocky structures that standard methods blur out.

## 1. The Problem with Smoothness

In Notebook 2 and 3, we used Tikhonov regularization ($||Lm||^2$). This mathematical trick assumes that the earth changes gradually. However, real geology is often discrete: fault lines, distinct rock layers, and intrusions have sharp boundaries.

Standard inversion methods "smear" these sharp boundaries, resulting in blurry images. To fix this, we need a **Prior** that understands distinct geological shapes.

## 2. Restricted Boltzmann Machines (RBM)

An RBM is a generative stochastic neural network that can learn a probability distribution over its set of inputs. It is an Energy-Based Model defined by a bipartite graph.

### 2.1. The Energy Function

The energy of the joint configuration of visible units ($v$) and hidden units ($h$) is defined as:

$$
E(\mathbf{v}, \mathbf{h}) = -\mathbf{v}^T \mathbf{W} \mathbf{h} - \mathbf{b}_v^T \mathbf{v} - \mathbf{b}_h^T \mathbf{h}
$$

**Where:**
* $\mathbf{v} \in \{0, 1\}^{N_v}$: The **Visible Vector**. In our case, this corresponds to the flattened conductivity model pixels ($m$).
* $\mathbf{h} \in \{0, 1\}^{N_h}$: The **Hidden Vector**. These are latent variables that detect features (e.g., edges, blocks).
* $\mathbf{W} \in \mathbb{R}^{N_v \times N_h}$: The **Weights Matrix**. It encodes the relationship between pixels and features. Trained from data.
* $\mathbf{b}_v$: Bias vector for the visible units.
* $\mathbf{b}_h$: Bias vector for the hidden units.

### 2.2. The Geological Force (Gradient)

To use the RBM as a prior in our inversion, we need the gradient of its **Free Energy** with respect to the visible units (the model). This tells us how to change the model to make it look more "geological".

The approximate gradient (based on Contrastive Divergence) is:

$$
\nabla_v \mathcal{F}(\mathbf{v}) \approx \mathbf{v} - \text{Reconstruct}(\mathbf{v})
$$

**Where:**
* $\mathbf{v}$: The current noisy model generated by P-bits.
* $\text{Reconstruct}(\mathbf{v})$: The model "imagined" by the RBM after passing $\mathbf{v}$ through the hidden layer and back.
* $\nabla_v \mathcal{F}$: The force vector. If the current model $\mathbf{v}$ is very different from what the RBM expects, this force will be large.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import joblib
from sklearn.neural_network import BernoulliRBM

sys.path.append(os.path.abspath('..'))

from pymtinv.mesh import create_padded_mesh
from pymtinv.forward import MT2DForward
from pymtinv.backward import MT2DGradient
from pymtinv.visualization import MT2DVisualizer
# We import the separate SciML functions
from pymtinv.utils import pbit_optimizer_with_rbm, tune_sciml_hyperparameters

## 3. Training the "Geological Brain"

First, we need to teach the RBM what geology looks like. We will generate a synthetic dataset of random blocks and layers, then train the RBM.

In [ ]:
# --- 3.1. Generate Synthetic Training Data ---
def generate_synthetic_geology(n_samples, ny, nz):
    data = np.zeros((n_samples, ny * nz))
    for i in range(n_samples):
        img = np.zeros((ny, nz))
        # Randomly add blocks or layers
        n_shapes = np.random.randint(1, 4)
        for _ in range(n_shapes):
            if np.random.rand() > 0.5: # Block
                w, h = np.random.randint(3, 8), np.random.randint(3, 8)
                y, z = np.random.randint(0, ny-w), np.random.randint(0, nz-h)
                img[y:y+w, z:z+h] = 1.0
            else: # Layer
                z_start = np.random.randint(0, nz-2)
                img[:, z_start:z_start+np.random.randint(1,4)] = 1.0
        data[i] = img.flatten()
    return data

# Define Mesh Size (Must match the inversion mesh core)
NY, NZ = 30, 25 
training_data = generate_synthetic_geology(2000, NY, NZ)

print(f"Generated 2000 synthetic geological models ({NY}x{NZ}).")

# --- 3.2. Train RBM ---
print("Training RBM (this acts as our geological prior)...")
rbm = BernoulliRBM(n_components=64, learning_rate=0.01, batch_size=10, n_iter=20, verbose=0)
rbm.fit(training_data)

# Save the trained model
RBM_FILENAME = "rbm_geology_prior.pkl"
joblib.dump(rbm, RBM_FILENAME)
print(f"RBM saved to {RBM_FILENAME}")

# Visualize what RBM learned (Weights)
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(rbm.components_[i].reshape(NY, NZ), cmap='viridis')
    plt.axis('off')
plt.suptitle("Learned RBM Features (Weights)")
plt.show()

## 4. Setting up the SciML Inversion

We will now invert a dataset generated from a **"Blocky"** model using the Hybrid (Physics + AI) optimizer.

In [ ]:
# 1. Setup Mesh (Matches RBM dimensions 30x25)
from pymtinv.mesh import TensorMesh
dy = np.ones(NY) * 500
dz = np.ones(NZ) * 250
mesh = TensorMesh(dy, dz)

# 2. True Model (Blocky)
sigma_true = np.ones((NY, NZ)) * 0.01
sigma_true[10:20, 10:15] = 1.0 # Sharp Conductive Block

# 3. Generate Data
freqs = np.logspace(2, -1, 5)
fwd = MT2DForward(mesh)
Z_true, _ = fwd.solve_te(freqs, sigma_true)

Z_obs = []
data_std = []
np.random.seed(42)
for f in freqs:
    val = Z_true[f]
    noise = 0.05 * np.abs(val) * (np.random.randn(*val.shape) + 1j*np.random.randn(*val.shape))
    Z_obs.append(val + noise)
    data_std.append(0.05 * np.abs(val))

grad_engine = MT2DGradient(fwd, Z_obs, data_std)
m_init = np.log10(np.ones((NY, NZ)) * 0.01).flatten()

## 5. Auto-Tuning AI Hyperparameters

In SciML, we have a new hyperparameter: **Alpha ($\\alpha$)**. This controls the weight of the RBM prior versus the Physics data.

* **Low Alpha:** Physics dominates (blurry result).
* **High Alpha:** AI dominates (hallucinations).

We use `tune_sciml_hyperparameters` to find the balance.

In [ ]:
print("Auto-tuning SciML parameters (Alpha & Learning Rate)...")
best_alpha, best_lr = tune_sciml_hyperparameters(
    grad_engine, freqs, mesh, m_init, 
    rbm_path=RBM_FILENAME
)

print(f"\nOptimal Parameters Found: Alpha={best_alpha}, LR={best_lr}")

## 6. Running the Hybrid Optimization

Now we run the `pbit_optimizer_with_rbm`. This function implements the combined update rule that merges Physics (Data Misfit) and AI (RBM Free Energy).

### 6.1. The Hybrid Update Equation

$$
m_{k+1} = m_k - \eta \left( \nabla \Phi_{data}(m_k) + \alpha \cdot \nabla \mathcal{F}_{RBM}(m_k) \right) + \sqrt{2 T_k} \cdot \xi
$$

**Where:**
* $m_k$: The conductivity model at step $k$.
* $\eta$: Learning rate (Step size).
* $\nabla \Phi_{data}$: The gradient from the Maxwell solver (Physics). Pushes model to fit data.
* $\nabla \mathcal{F}_{RBM}$: The gradient from the RBM (AI). Pushes model to look geological.
* $\alpha$: Weighting factor (The "SciML coefficient") determined by auto-tuning.
* $T_k$: Annealing temperature at step $k$.
* $\xi$: Random Gaussian noise.

We also use **Averaging** (averaging the last 1000 steps) to clean up the stochastic noise and reveal the sharp structure.

In [ ]:
print("Starting Hybrid Inversion...")
m_final, hist, phi = pbit_optimizer_with_rbm(
    grad_engine, 
    m_init, 
    freqs, 
    mesh, 
    n_steps=4000, 
    lr=best_lr, 
    t_start=0.1, 
    t_end=0.0001, 
    rbm_path=RBM_FILENAME,
    alpha=best_alpha,
    average_last_steps=1000 # Clean up the noise
)

m_final_2d = m_final.reshape((NY, NZ))

## 7. Results: Physics vs. AI

Compare the True Model with the SciML Recovered Model. Observe how the RBM has helped the optimizer to make "decisions" and form sharp boundaries, effectively acting as a discrete geological regularizer.

In [ ]:
plt.figure(figsize=(12, 5))

ax1 = plt.subplot(1, 3, 1)
MT2DVisualizer.plot_model(mesh, np.log10(sigma_true), ax=ax1, title="True Model (Blocky)")

ax2 = plt.subplot(1, 3, 2)
MT2DVisualizer.plot_model(mesh, m_final_2d, ax=ax2, title=f"SciML Result (Phi={phi:.1f})")

ax3 = plt.subplot(1, 3, 3)
plt.plot(hist)
plt.title("Convergence")
plt.xlabel("Iteration")
plt.grid(True)

plt.tight_layout()
plt.show()